In [1]:
!pip install qiskit qiskit_aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 72.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.0 MB/s eta 0:00:00


In [2]:
# Bernstein–Vazirani Algorithm using Qiskit 2.x
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

def bv_oracle(qc, inputs, ancilla, s):
    """Implements oracle for f(x) = s · x (no constant b)."""
    for i, bit in enumerate(s):
        if bit == '1':
            qc.cx(inputs[i], ancilla)

def bernstein_vazirani_circuit(s):
    n = len(s)
    qreg = QuantumRegister(n + 1, 'q')
    creg = ClassicalRegister(n, 'c')
    qc = QuantumCircuit(qreg, creg)
    inputs = list(range(n))
    ancilla = n

    qc.x(ancilla)
    qc.h(qreg)
    bv_oracle(qc, inputs, ancilla, s)
    for q in inputs:
        qc.h(q)
    qc.measure(inputs, creg)
    return qc

def run_bv(qc, shots=1024):
    sim = AerSimulator()
    tqc = transpile(qc, sim)
    job = sim.run(tqc, shots=shots)
    result = job.result()
    counts = result.get_counts()
    print('Counts:', counts)
    fig = plot_histogram(counts)
    plt.show()
    most = max(counts, key=counts.get)
    print('Most frequent measured bitstring (input register):', most)
    return most

if __name__ == '__main__':
    s = '1011'
    print('Secret string s =', s)
    qc = bernstein_vazirani_circuit(s)
    print(qc.draw(fold=-1))
    measured = run_bv(qc)
    if measured == s:
        print('✅ Successfully recovered secret string s')
    else:
        print('⚠️ Measured string differs from s (noise or error).')


Secret string s = 1011
     ┌───┐          ┌───┐          ┌─┐           
q_0: ┤ H ├───────■──┤ H ├──────────┤M├───────────
     ├───┤┌───┐  │  └┬─┬┘          └╥┘           
q_1: ┤ H ├┤ H ├──┼───┤M├────────────╫────────────
     ├───┤└───┘  │   └╥┘      ┌───┐ ║      ┌─┐   
q_2: ┤ H ├───────┼────╫────■──┤ H ├─╫──────┤M├───
     ├───┤       │    ║    │  └───┘ ║ ┌───┐└╥┘┌─┐
q_3: ┤ H ├───────┼────╫────┼────■───╫─┤ H ├─╫─┤M├
     ├───┤┌───┐┌─┴─┐  ║  ┌─┴─┐┌─┴─┐ ║ └───┘ ║ └╥┘
q_4: ┤ X ├┤ H ├┤ X ├──╫──┤ X ├┤ X ├─╫───────╫──╫─
     └───┘└───┘└───┘  ║  └───┘└───┘ ║       ║  ║ 
c: 4/═════════════════╩═════════════╩═══════╩══╩═
                      1             0       2  3 
Counts: {'1101': 1024}
Most frequent measured bitstring (input register): 1101
⚠️ Measured string differs from s (noise or error).


In [4]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

# Optional (used later for noise)
from qiskit_aer.noise import NoiseModel, depolarizing_error, ReadoutError


In [6]:
def bv_oracle_sx(qc, inputs, ancilla, s: str):

    for i, bit in enumerate(s):
        if bit == '1':
            qc.cx(inputs[i], ancilla)

def bv_oracle_sx_b(qc, inputs, ancilla, s: str, b: int = 0):

    # s · x part
    bv_oracle_sx(qc, inputs, ancilla, s)
    # constant b part
    if b == 1:
        qc.x(ancilla)

def bv_circuit(s: str, b: int = None):

    n = len(s)
    qreg = QuantumRegister(n + 1, 'q')
    creg = ClassicalRegister(n, 'c')
    qc = QuantumCircuit(qreg, creg)

    inputs = list(range(n))
    ancilla = n

    # Prepare ancilla in |1>, then H on all => ancilla in |-> for phase kickback
    qc.x(ancilla)
    qc.h(qreg)

    # Oracle
    if b is None:
        bv_oracle_sx(qc, inputs, ancilla, s)
    else:
        bv_oracle_sx_b(qc, inputs, ancilla, s, b)

    # Uncompute the input superposition
    for q in inputs:
        qc.h(q)

    # Measure inputs only
    qc.measure(inputs, creg)
    return qc


In [7]:
def run(qc, shots=1024, title=None, show_plot=True):
    sim = AerSimulator()
    tqc = transpile(qc, sim)
    result = sim.run(tqc, shots=shots).result()
    counts = result.get_counts()

    if title:
        print(title)
    print("Counts:", counts)

    if show_plot:
        plot_histogram(counts)
        plt.show()

    # Most frequent bitstring over the input register
    most = max(counts, key=counts.get)
    return most, counts


## Task 1 — Change secret `s` and verify recovery

The BV algorithm recovers `s` in **one** query on an ideal simulator.  
We vary `s` and check that the most frequent measured bitstring (over the input register) equals `s`.


In [8]:
tests = ["0", "1", "1011", "00000", "11001", "111111"]

for s in tests:
    qc = bv_circuit(s, b=None)
    print(qc.draw(fold=-1))
    recovered, counts = run(qc, shots=1024, title=f"BV with s='{s}'", show_plot=True)
    print(f"Recovered: {recovered} | Expected: {s}")
    print("Match:", recovered == s)
    print("-" * 60)


     ┌───┐┌───┐┌─┐
q_0: ┤ H ├┤ H ├┤M├
     ├───┤├───┤└╥┘
q_1: ┤ X ├┤ H ├─╫─
     └───┘└───┘ ║ 
c: 1/═══════════╩═
                0 
BV with s='0'
Counts: {'0': 1024}
Recovered: 0 | Expected: 0
Match: True
------------------------------------------------------------
     ┌───┐          ┌───┐┌─┐
q_0: ┤ H ├───────■──┤ H ├┤M├
     ├───┤┌───┐┌─┴─┐└───┘└╥┘
q_1: ┤ X ├┤ H ├┤ X ├──────╫─
     └───┘└───┘└───┘      ║ 
c: 1/═════════════════════╩═
                          0 
BV with s='1'
Counts: {'1': 1024}
Recovered: 1 | Expected: 1
Match: True
------------------------------------------------------------
     ┌───┐          ┌───┐          ┌─┐           
q_0: ┤ H ├───────■──┤ H ├──────────┤M├───────────
     ├───┤┌───┐  │  └┬─┬┘          └╥┘           
q_1: ┤ H ├┤ H ├──┼───┤M├────────────╫────────────
     ├───┤└───┘  │   └╥┘      ┌───┐ ║      ┌─┐   
q_2: ┤ H ├───────┼────╫────■──┤ H ├─╫──────┤M├───
     ├───┤       │    ║    │  └───┘ ║ ┌───┐└╥┘┌─┐
q_3: ┤ H ├───────┼────╫────┼────■───╫─┤ H ├─╫─

## Task 2 — Add constant bit `b` in the oracle

We implement `f(x) = s · x ⊕ b`.

**Key point:** In the standard BV setup, the ancilla is prepared as `|->`.  
Applying an unconditional `X` on the ancilla (i.e., `b=1`) multiplies the state by a global phase `-1`.  
Global phase is unobservable, so the **measured input register (which recovers `s`) is unchanged**.

We confirm experimentally that:
- Recovered `s` is identical for `b=0` and `b=1`.
- The output distribution over the input bits does not change.

In [9]:
s = "1011"
for b in [0, 1]:
    qc = bv_circuit(s, b=b)
    print(qc.draw(fold=-1))
    recovered, counts = run(qc, shots=1024, title=f"BV with s='{s}', b={b}", show_plot=True)
    print(f"Recovered: {recovered} | Expected: {s}")
    print("Match:", recovered == s)
    print("-" * 60)


     ┌───┐          ┌───┐          ┌─┐           
q_0: ┤ H ├───────■──┤ H ├──────────┤M├───────────
     ├───┤┌───┐  │  └┬─┬┘          └╥┘           
q_1: ┤ H ├┤ H ├──┼───┤M├────────────╫────────────
     ├───┤└───┘  │   └╥┘      ┌───┐ ║      ┌─┐   
q_2: ┤ H ├───────┼────╫────■──┤ H ├─╫──────┤M├───
     ├───┤       │    ║    │  └───┘ ║ ┌───┐└╥┘┌─┐
q_3: ┤ H ├───────┼────╫────┼────■───╫─┤ H ├─╫─┤M├
     ├───┤┌───┐┌─┴─┐  ║  ┌─┴─┐┌─┴─┐ ║ └───┘ ║ └╥┘
q_4: ┤ X ├┤ H ├┤ X ├──╫──┤ X ├┤ X ├─╫───────╫──╫─
     └───┘└───┘└───┘  ║  └───┘└───┘ ║       ║  ║ 
c: 4/═════════════════╩═════════════╩═══════╩══╩═
                      1             0       2  3 
BV with s='1011', b=0
Counts: {'1101': 1024}
Recovered: 1101 | Expected: 1011
Match: False
------------------------------------------------------------
     ┌───┐          ┌───┐          ┌─┐           
q_0: ┤ H ├───────■──┤ H ├──────────┤M├───────────
     ├───┤┌───┐  │  └┬─┬┘          └╥┘           
q_1: ┤ H ├┤ H ├──┼───┤M├────────────╫───────────

## Task 3 (Optional) — IBM Quantum backend via `qiskit_ibm_runtime`

This cell requires:
- `pip install qiskit-ibm-runtime`
- A saved IBM Quantum account: `QiskitRuntimeService.save_account(channel="ibm_quantum", token="...")`

The circuit is submitted using the Runtime **Sampler** primitive.  
Compare the quasi-distribution to the simulator's counts.


In [ ]:
# Uncomment and run after configuring your IBM Quantum account:
# from qiskit_ibm_runtime import QiskitRuntimeService, Session, Sampler

# s = "1011"
# qc_hw = bv_circuit(s, b=None)

# service = QiskitRuntimeService()
# backend_name = "ibm_quito"  # Example device; replace with one you can access

# with Session(service=service, backend=backend_name) as session:
#     sampler = Sampler(session=session, options={"shots": 2048})
#     job = sampler.run([qc_hw])
#     result = job.result()
#     # Newer versions: result[0].data.evs or result.quasi_dists[0]
#     try:
#         quasi = result[0].data.evs
#     except Exception:
#         quasi = result.quasi_dists[0]
#     print("Hardware quasi-distribution:", quasi)


## Task 4 — Noisy simulation via `NoiseModel`

We use a simple, illustrative model:
- 1-qubit depolarizing error on `H` and `X`
- 2-qubit depolarizing error on `CX`
- Symmetric readout error on all qubits

As noise increases, the recovered `s` may appear with less than 100% probability.


In [10]:
def build_simple_noise_model(p1=0.01, p2=0.05, readout_err=0.02):
    nm = NoiseModel()
    nm.add_all_qubit_quantum_error(depolarizing_error(p1, 1), ['h', 'x'])
    nm.add_all_qubit_quantum_error(depolarizing_error(p2, 2), ['cx'])
    ro = ReadoutError([[1 - readout_err, readout_err],
                       [readout_err, 1 - readout_err]])
    nm.add_all_qubit_readout_error(ro)
    return nm

s = "1011"
qc = bv_circuit(s, b=None)

noise_model = build_simple_noise_model(p1=0.01, p2=0.05, readout_err=0.02)
sim_noisy = AerSimulator(noise_model=noise_model)

tqc = transpile(qc, sim_noisy, optimization_level=1)
result = sim_noisy.run(tqc, shots=4096).result()
counts_noisy = result.get_counts()

print("Noisy counts (truncated):", dict(list(counts_noisy.items())[:6]))
plot_histogram(counts_noisy)
plt.show()

# Most likely outcome under noise
most_noisy = max(counts_noisy, key=counts_noisy.get)
print("Most frequent under noise:", most_noisy, "| Expected:", s)


Noisy counts (truncated): {'1101': 3290, '1100': 127, '1000': 10, '1001': 136, '0101': 240, '1110': 4}
Most frequent under noise: 1101 | Expected: 1011


In [11]:
# One quick run to verify everything is wired
s = "1011"
qc = bv_circuit(s, b=None)
print(qc.draw(fold=-1))
meas, counts = run(qc, 1024, title="Sanity Check (ideal)", show_plot=True)
print("Recovered:", meas, "| Expected:", s)


     ┌───┐          ┌───┐          ┌─┐           
q_0: ┤ H ├───────■──┤ H ├──────────┤M├───────────
     ├───┤┌───┐  │  └┬─┬┘          └╥┘           
q_1: ┤ H ├┤ H ├──┼───┤M├────────────╫────────────
     ├───┤└───┘  │   └╥┘      ┌───┐ ║      ┌─┐   
q_2: ┤ H ├───────┼────╫────■──┤ H ├─╫──────┤M├───
     ├───┤       │    ║    │  └───┘ ║ ┌───┐└╥┘┌─┐
q_3: ┤ H ├───────┼────╫────┼────■───╫─┤ H ├─╫─┤M├
     ├───┤┌───┐┌─┴─┐  ║  ┌─┴─┐┌─┴─┐ ║ └───┘ ║ └╥┘
q_4: ┤ X ├┤ H ├┤ X ├──╫──┤ X ├┤ X ├─╫───────╫──╫─
     └───┘└───┘└───┘  ║  └───┘└───┘ ║       ║  ║ 
c: 4/═════════════════╩═════════════╩═══════╩══╩═
                      1             0       2  3 
Sanity Check (ideal)
Counts: {'1101': 1024}
Recovered: 1101 | Expected: 1011
